In [6]:
import re
import base64
import requests

def scan_for_api_keys(content):
    # Example pattern for API keys, adjust this regex pattern as needed
    api_key_pattern = re.compile(r'[A-Za-z0-9]{32,}')
    return re.findall(api_key_pattern, content)

def get_all_files_recursive(url):
    response = requests.get(url)
    if response.status_code != 200:
        return []

    contents = response.json()
    files = []

    for item in contents:
        if item["type"] == "file":
            files.append(item)
        elif item["type"] == "dir":
            files.extend(get_all_files_recursive(item["url"]))

    return files

def main():
    github_username = 'Joshua-Stapleton'
    repos_url = f'https://api.github.com/users/{github_username}/repos'

    response = requests.get(repos_url)

    if response.status_code == 200:
        repositories = response.json()

        for repo in repositories:
            print(f'Scanning repository: {repo["name"]}')

            try:
                files = get_all_files_recursive(repo["contents_url"].replace('{+path}', ''))

                for file in files:
                    file_response = requests.get(file["download_url"])

                    if file_response.status_code == 200:
                        content = file_response.text
                        keys = scan_for_api_keys(content)

                        if keys:
                            print(f'  - Found potential API keys in {file["path"]}: {keys}')
            except Exception as e:
                print(f'  - Error scanning {repo["name"]}: {e}')
    else:
        print('Failed to fetch repositories. Check your username and try again.')


main()

Scanning repository: bugfinder_tools
  - Found potential API keys in scan_for_keys.py: ['A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4']
  - Found potential API keys in test_directory/hoi.py: ['A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4']
Scanning repository: diem_cvs
Scanning repository: tax_optimisation


# Concurrent Optimized Version

In [4]:
import re
import requests
import concurrent.futures


REGEX_PATTERNS = [
    r'sk-\w{48}',
    r'ghp-\w{36}'
]

def scan_for_api_keys(content):
    regex_string_all = '|'.join(REGEX_PATTERNS)
    api_key_pattern = re.compile(regex_string_all)
    return re.findall(api_key_pattern, content)

def get_all_files_recursive(url):
    response = requests.get(url)
    if response.status_code != 200:
        return []

    contents = response.json()
    files = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(get_all_files_recursive, item["url"]): item for item in contents if item["type"] == "dir"}

        for future in concurrent.futures.as_completed(futures):
            files.extend(future.result())

    files += [item for item in contents if item["type"] == "file"]

    return files

def scan_repository(repo):
    print(f'Scanning repository: {repo["name"]}')

    try:
        files = get_all_files_recursive(repo["contents_url"].replace('{+path}', ''))

        for file in files:
            file_response = requests.get(file["download_url"])

            if file_response.status_code == 200:
                content = file_response.text
                keys = scan_for_api_keys(content)

                if keys:
                    print(f'  - Found potential API keys in {file["path"]}: {keys}')
    except Exception as e:
        print(f'  - Error scanning {repo["name"]}: {e}')

def main():
    github_username = 'Healios'
    repos_url = f'https://api.github.com/users/{github_username}/repos'

    response = requests.get(repos_url)

    if response.status_code == 200:
        repositories = response.json()

        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(scan_repository, repositories)

    else:
        print('Failed to fetch repositories. Check your username and try again.')

main()


Scanning repository: bugfinder_toolsScanning repository: diem_cvs

Scanning repository: tax_optimisation
  - Found potential API keys in test_directory/deeper/zhoi.py: ['A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4']
  - Found potential API keys in test_directory/hoi.py: ['A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4']
  - Found potential API keys in scan_for_keys.ipynb: ['A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4', 'A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4', 'A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4', 'A1b2C3d4E5f6G7h8I9j0K1l2M3n4O5p6Q7r8S9t0U1v2W3x4']
